In [ ]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images

In [ ]:
!unzip -q /content/deepfake-and-real-images.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Datagen = ImageDataGenerator(rescale=1./255)

Train_generator = Datagen.flow_from_directory(
    "/kaggle/input/deepfake-and-real-images/Dataset/Train",
    target_size = (224,224),
    batch_size = 64,
    class_mode = "binary",
    shuffle = True
)

Valid_generator = Datagen.flow_from_directory(
    "/kaggle/input/deepfake-and-real-images/Dataset/Validation",
    target_size = (224,224),
    batch_size = 64,
    class_mode = "binary",
    shuffle = True
)


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


In [ ]:
# Model

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, add, BatchNormalization, Dropout, GlobalAveragePooling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

model = InceptionV3(
    include_top = False,
    weights = "imagenet",
    input_shape = (224,224,3)
)

for layer in model.layers:
  layer.trainable = False

# Get the output of the InceptionV3 model
x = model.output

# Add new layers on top of the InceptionV3 output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(128, activation='relu', kernel_regularizer=l2(0.05))(x)  # Dense layer with 128 units
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
x = Dense(64, activation='relu', kernel_regularizer=l2(0.05))(x)  # Dense layer with 64 units
x = BatchNormalization()(x)  # Batch Normalization
x = Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation

# Create a new model with the specified inputs and outputs
model = Model(inputs=model.input, outputs=x)

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# import precision, Recall
from tensorflow.keras.metrics import Precision, Recall
# from tensorflow_addons.metrics import F1Score  # If using TensorFlow Addons

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss="binary_crossentropy",
    metrics=["accuracy", Precision(), Recall()]
)
# Callbacks
earlystopping = EarlyStopping(
    monitor="val_loss",
    verbose=1,
    patience = 3,
    restore_best_weights=True
)

modelcheckpoint = ModelCheckpoint(
    "model.keras",
    monitor="val_loss",
    verbose=1,
    save_best_only=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    verbose=1,
    min_lr=1e-6,
    patience = 1
)

# Training the model
history = model.fit(
    Train_generator,
    epochs=30,
    validation_data=Valid_generator,
    verbose=1,
    callbacks=[earlystopping, modelcheckpoint, reduce_lr]
)

In [ ]:
# Visulaize

import matplotlib.pyplot as plt

plt.plot(history.history["loss"], label = "Train Loss")
plt.plot(history.history["val_loss"], label = "Validation Loss")
plt.legend()
plt.show()


In [ ]:
plt.plot(history.history["accuracy"], label = "Train Accuracy")
plt.plot(history.history["val_accuracy"], label = "Validation Accuracy")
plt.legend()
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Prepare the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "/kaggle/input/deepfake-and-real-images/Dataset/Test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Binary classification
    shuffle=False  # Important for preserving the order of labels
)

# Predict on the test set
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype(int).flatten()  # Convert probabilities to class labels

# Get the true labels
true_classes = test_generator.classes

# Calculate confusion matrix and classification report
cm = confusion_matrix(true_classes, predicted_classes)
report = classification_report(true_classes, predicted_classes, target_names=['Fake', 'Real'])

print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

from sklearn.metrics import f1_score

# Assuming you already have true_classes and predicted_classes from your model
f1 = f1_score(true_classes, predicted_classes)
print("F1 Score:", f1)
